In [28]:
import cv2
import numpy as np
import pyautogui
from ultralytics import YOLO

In [ ]:
modelPath = "Yolos/Honeycomb/train43/weights/best.pt"
model = YOLO(modelPath)

In [30]:
threshold = 0.3

In [ ]:
screenWidth, screenHeight = pyautogui.size()

region = (0, 0, screenWidth, screenHeight)

windowName = "YOLO Detection on Screen"
cv2.namedWindow(windowName, cv2.WINDOW_NORMAL)
cv2.resizeWindow(windowName, 400, 300)

while True:
    screenshot = pyautogui.screenshot(region=region)
    frame = np.array(screenshot)

    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    results = model.predict(source=frame, conf=threshold, verbose=False)

    detections = results[0].boxes.data.cpu().numpy()
    classCounts = {}
    for detection in detections:
        classId = int(detection[5])
        classCounts[classId] = classCounts.get(classId, 0) + 1

    annotatedFrame = results[0].plot()

    yOffset = 20
    for classId, count in classCounts.items():
        class_name = model.names[classId]
        text = f"{class_name}: {count}"
        cv2.putText(
            annotatedFrame,
            text,
            (10, yOffset),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.7,
            (0, 255, 0),
            2,
            cv2.LINE_AA,
        )
        yOffset += 30
    cv2.imshow(windowName, annotatedFrame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cv2.destroyAllWindows()